In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd '/content/gdrive/My Drive/LDS0_K287_Online_PhamMinhQuang/Project2/'

Mounted at /content/gdrive
/content/gdrive/My Drive/LDS0_K287_Online_PhamMinhQuang/Project2


In [ ]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 48.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re

In [ ]:
STOP_WORD_FILE = 'vietnamese-stopwords.txt'

In [ ]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()
stop_words = stop_words.split('\n')

In [ ]:
df = pd.read_csv('ProductRaw.csv')[['name', 'description']]

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df['products'] = df['name'] + ' ' + df['description']

In [ ]:
# ở source này em chọn name và description để phân tích
df = df[['products']].astype(str)

In [ ]:
df.head()

products
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời gian pin - Thời gian nghe nhạc liên tục từ 2.5-4h - Thời gian sạc đầy chỉ khoảng 60p - Thời gian chờ lên tới 140 giờ\nBluetooth 5\nThương hiệu OEM\nXuất xứ thương hiệu Trung Quốc\nĐộ nhạy cảm biến vân tay\nModel i12\nLoại Jack cắm USB Cable\nTrọng lượng 300g\nThời gian sử dụng\n- Thời gian nghe nhạc liên tục từ 2.5-4h\nSKU 4096608751631\nMÔ TẢ SẢN PHẨM\nINPOD 12 là phiên bản nâng cấp mới nhất , tai nghe Bluetooth 5.0 có thiết kế tỉ lệ chuẩn 1:1 với tai Airpod chính hãng\nLược bỏ nút bấm trên thân tai thay vào đó là nút cảm ứng sử dụng dễ dàng và thuận tiện hơn\nThao tác nhận cuộc gọi đến, chuyển nhạc...một cách dễ dàng chỉ bằng 1 chạm\nBluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- Bluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- 2 tai nghe tự kết nối với nhau khi được lấy ra từ dock sạc\n- Chất âm và thời lượng pin được cải thiện tối ưu\n- Dock sạc tiện lợi cho tới 4 lần sạc đầy pin tai nghe\n-Các thiết bị tương thích: các thiết bị Bluetooth như cho Xiaomi, Samsung, Apple, Huawei, OPPO, vivo, Lenovo, tablet, v. v. \n- Thời gian nghe nhạc liên tục từ 2.5-4h\n- Thời gian sạc đầy chỉ khoảng 60p\n- Thời gian chờ lên tới 140 giờ\nGiá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. Tuy 

### data exploration

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   products  4404 non-null   object
dtypes: object(1)
memory usage: 34.5+ KB


In [ ]:
df.duplicated().sum()

33

In [ ]:
#drop dup
df.drop_duplicates(subset=['products'], inplace = True, ignore_index = True)

In [ ]:
# word_tokenize
df["products_wt"]=df["products"].apply(lambda x: word_tokenize(x, format="text"))

In [ ]:
df.head()

products  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời gian pin - Thời gian nghe nhạc liên tục từ 2.5-4h - Thời gian sạc đầy chỉ khoảng 60p - Thời gian chờ lên tới 140 giờ\nBluetooth 5\nThương hiệu OEM\nXuất xứ thương hiệu Trung Quốc\nĐộ nhạy cảm biến vân tay\nModel i12\nLoại Jack cắm USB Cable\nTrọng lượng 300g\nThời gian sử dụng\n- Thời gian nghe nhạc liên tục từ 2.5-4h\nSKU 4096608751631\nMÔ TẢ SẢN PHẨM\nINPOD 12 là phiên bản nâng cấp mới nhất , tai nghe Bluetooth 5.0 có thiết kế tỉ lệ chuẩn 1:1 với tai Airpod chính hãng\nLược bỏ nút bấm trên thân tai thay vào đó là nút cảm ứng sử dụng dễ dàng và thuận tiện hơn\nThao tác nhận cuộc gọi đến, chuyển nhạc...một cách dễ dàng chỉ bằng 1 chạm\nBluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- Bluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- 2 tai nghe tự kết nối với nhau khi được lấy ra từ dock sạc\n- Chất âm và thời lượng pin được cải thiện tối ưu\n- Dock sạc tiện lợi cho tới 4 lần sạc đầy pin tai nghe\n-Các thiết bị tương thích: các thiết bị Bluetooth như cho Xiaomi, Samsung, Apple, Huawei, OPPO, vivo, Lenovo, tablet, v. v. \n- Thời gian nghe nhạc liên tục từ 2.5-4h\n- Thời gian sạc đầy chỉ khoảng 60p\n- Thời gian chờ lên tới 140 giờ\nGiá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. T

### Genshim

In [ ]:
# Tokenize(split) the sentences into words
products_gem = [[text for text in x.split()] for x in df.products_wt]

In [ ]:
products_gem[:2]

[['Tai',
  'nghe',
  'Bluetooth_Inpods',
  '12',
  '-',
  'Cảm_biến',
  'vân',
  'tay',
  ',',
  'chống',
  'nước',
  ',',
  'màu_sắc',
  'đa_dạng',
  '-',
  '5',
  'màu_sắc',
  'lựa_chọn',
  'THÔNG_TIN',
  'CHI_TIẾT',
  'Dung_lượng',
  'pin',
  '300',
  'Thời_gian',
  'pin',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  '-',
  'Thời_gian',
  'sạc',
  'đầy',
  'chỉ',
  'khoảng',
  '60',
  'p',
  '-',
  'Thời_gian',
  'chờ',
  'lên',
  'tới',
  '140',
  'giờ',
  'Bluetooth',
  '5',
  'Thương_hiệu',
  'OEM',
  'Xuất_xứ',
  'thương_hiệu',
  'Trung_Quốc',
  'Độ',
  'nhạy_cảm_biến',
  'vân',
  'tay',
  'Model_i12',
  'Loại',
  'Jack_cắm',
  'USB_Cable',
  'Trọng_lượng',
  '300',
  'g',
  'Thời_gian',
  'sử_dụng',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  'SKU_4096608751631',
  'MÔ_TẢ',
  'SẢN_PHẨM',
  'INPOD',
  '12',
  'là',
  'phiên_bản',
  'nâng_cấp',
  'mới',
  'nhất',
  ',',
  'tai',

In [ ]:
len(products_gem)

4371

In [ ]:
# phần này viết thành function, bỏ bớt những từ ko cần thiết
def removeUnnecessariness(products_gem):
  products_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in products_gem] # bỏ số
  products_gem_re = [[re.sub('\n',' ', e) for e in text] for text in products_gem_re]
  products_gem_re = [[t.lower() for t in text ] for text in  products_gem_re]
  products_gem_re = [[t for t in text if not t in ['thông_tin', 'chi_tiết', '', ' ', ',', '.', '...', '-',':', ';', '?', '%', '(', ')', '+', '/', 'g', 'ml',
  'tới','tùy','vào','từng','loại','mà','như','hàng','có','không', 'dây','hơn','với','và','cho','có','rất','nhau','bạn','sẽ','theo']] for text in  products_gem_re] # ký tự đặc biệt
  products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem_re] # stopword
  return products_gem_re
products_gem_re = removeUnnecessariness(products_gem)

In [ ]:
dictionary = corpora.Dictionary(products_gem_re)

In [ ]:
dictionary.token2id

{'apple': 0,
 'bao_gồm': 1,
 'bluetooth': 2,
 'bluetooth_.': 3,
 'bluetooth_inpods': 4,
 'bấm': 5,
 'chi_phí': 6,
 'chuyển_nhạc': 7,
 'chuẩn': 8,
 'chạm': 9,
 'chất_âm': 10,
 'chống': 11,
 'chờ': 12,
 'cuộc_gọi': 13,
 'cải_thiện': 14,
 'cảm_biến': 15,
 'cảm_ứng': 16,
 'cồng_kềnh': 17,
 'dock': 18,
 'dung_lượng': 19,
 'dễ_dàng': 20,
 'giao': 21,
 'giá': 22,
 'h': 23,
 'hiện_hành': 24,
 'huawei': 25,
 'hãng': 26,
 'inpod': 27,
 'jack_cắm': 28,
 'kết_nối': 29,
 'lenovo': 30,
 'liên_tục': 31,
 'luật': 32,
 'lược': 33,
 'lựa_chọn': 34,
 'model_i': 35,
 'màu_sắc': 36,
 'mô_tả': 37,
 'nhạc': 38,
 'nhạy_cảm_biến': 39,
 'nâng_cấp': 40,
 'nút': 41,
 'oem': 42,
 'oppo': 43,
 'p': 44,
 'phiên_bản': 45,
 'phát_sinh': 46,
 'phí': 47,
 'phương_thức': 48,
 'phụ_phí': 49,
 'pin': 50,
 'pin_tai': 51,
 'samsung': 52,
 'sku_': 53,
 'sạc': 54,
 'sản_phẩm': 55,
 'tablet': 56,
 'tai': 57,
 'tai_airpod': 58,
 'thao_tác': 59,
 'thay': 60,
 'thiết_bị': 61,
 'thiết_kế': 62,
 'thuận_tiện': 63,
 'thuế': 64,
 'thân

In [ ]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)
feature_cnt

45713

In [ ]:
# Obtain corpus based on dictionary (dense matrix)
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [ ]:
corpus[:1]

[[(0, 1),
  (1, 1),
  (2, 2),
  (3, 3),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 2),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 2),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 3),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 3),
  (30, 1),
  (31, 3),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 2),
  (37, 1),
  (38, 3),
  (39, 1),
  (40, 1),
  (41, 2),
  (42, 1),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 3),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 5),
  (55, 3),
  (56, 1),
  (57, 3),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 2),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 2),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 2),
  (77, 1),
  (78, 2),
  (79, 2),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 2)]]

In [ ]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)
# tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)

### test 1 id bất kỳ lấy ra n similarities

In [ ]:
def findSimilarityExisting (n, idx, tfidf, index, corpus, df):
  my_dict = {}
  sim = index[tfidf[corpus[idx]]]
  for i in range(len(sim)):
    my_dict.update({i:sim[i]})
  max_value = sorted(my_dict, key=my_dict.get, reverse=True)[1:n+1]
  for i in range(0, len(max_value)):
    print('match {}: '.format(i+1), df.values[max_value[i]])

In [ ]:
findSimilarityExisting (3, 55, tfidf, index, corpus, df)

match 1:  ['Loa tép AT46, hàng chính hãng THÔNG TIN CHI TIẾT\nThương hiệu Thumper\nKích thước\ndài x rộng x cao: 22 x 11 x 15cm\nModel at46\nSKU 7043839263461\nMÔ TẢ SẢN PHẨM\nCông suất âm thanh (W): 250W\nTần số thấp nhất (Hz):1.000Hz\nTần số cao nhất (kHz):20.000Hz\nMàu sắc: Màu đen\nTrở kháng(Ω): 8Ω\nĐộ nhạy: 92 dB\nXuất xứ Việt Nam\nThông số chi tiết:\nCông suất âm thanh (W): 250W\nTần số thấp nhất (Hz):1.000Hz\nTần số cao nhất (kHz):20.000Hz\nMàu sắc: Màu đen\nTrở kháng(Ω): 8Ω\nĐộ nhạy: 92 dB\nXuất xứ Việt Nam\n\n\n\n\n\n\n\n\n  Thông số chi tiết:\nCông suất âm thanh (W): 250W\nTần số thấp nhất (Hz):1.000Hz\nTần số cao nhất (kHz):20.000Hz\nMàu sắc: Màu đen\nTrở kháng(Ω): 8Ω\nĐộ nhạy: 92 dB\nXuất xứ Việt Nam\ntép rời at46\nGiá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. Tuy nhiên tuỳ vào từng loại sản phẩm hoặc phương thức, địa chỉ giao hàng mà có thể phát sinh thêm chi phí khác như phí vận chuyển, phụ phí hàng cồng kềnh, ...\n'
 'Loa tép AT46 , hàng chính hãng THÔNG_TI

### test tên mặt hàng bất kỳ

In [ ]:
df = pd.read_csv('ProductRaw.csv')[['name', 'description']]
df['products'] = df['name'] + ' ' + df['description']
df = df[['products']].astype(str)
df.drop_duplicates(subset=['products'], inplace = True, ignore_index = True)

In [ ]:
def findSimilarityArbitrary(input_text, df):
  df_newline = pd.DataFrame({"products":[input_text]})
  df = df.append(df_newline, ignore_index = True)
  df["products_wt"]=df["products"].apply(lambda x: word_tokenize(x, format="text"))
  products_gem = [[text for text in x.split()] for x in df.products_wt]
  products_gem_re = removeUnnecessariness(products_gem)
  dictionary = corpora.Dictionary(products_gem_re)
  feature_cnt = len(dictionary.token2id)
  corpus = [dictionary.doc2bow(text) for text in products_gem_re]
  tfidf = models.TfidfModel(corpus)
  index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
  findSimilarityExisting(4, len(df)-1, tfidf, index, corpus, df)

In [ ]:
findSimilarityArbitrary('''
Điện thoại Samsung Galaxy A53 5G (8GB/128GB) - Hàng chính hãng
''', df)

<ipython-input-31-b48e070d44c7>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_newline, ignore_index = True)


match 1:  ['Điện Thoại Samsung Galaxy S21 Ultra 5G (12GB/128GB) - Hàng Chính Hãng THÔNG TIN CHI TIẾT\nDung lượng pin 5000 mAh\nBluetooth Bluetooth 5.1\nThương hiệu Samsung\nXuất xứ thương hiệu Hàn Quốc\nChip set Exynos 2100\nNFC Có\nFM radio Không\nGhi âm Có\nSố sim 2\nLoại pin 5.000 mAh\nNghe nhạc Có\nXuất xứ Hàn Quốc\nCổng sạc USB Type-C 3.2\nRAM 16 GB\nROM 512 GB\nKích thước màn hình 6.8 inch\nXem phim Có\nSKU 8549842564143\nMÔ TẢ SẢN PHẨM\nThiết kế ấn tượng\nTheo như những hình ảnh rò rỉ lan truyền qua nhiều nguồn, thì dòng sản phẩm mới sắp ra mắt Samsung Galaxy S21 Ultra sẽ sở hữu khung vỏ kim loại và hai mặt kính trước sau nịnh mắt – ngôn ngữ thiết kế chủ đạo trên các dòng smartphone cao cấp nhất hiện tại.\nMàn hình Full HD+, tần số quét 120Hz mượt mà\nTheo các nguồn tin cho hay thì Samsung Galaxy S21 Ultra được trang bị màn hình full HD+ siêu sắc nét với tần số quét 120Hz mượt mà.\nHiệu năng mạnh mẽ\nSamsung Galaxy S21 Ultra với chip Exynos 2100 hoặc Snapdragon 875 cho tốc độ

Nhận xét: kết quả recommend khá tốt, ra đúng nhãn hiệu tương tự và dòng máy tương tự cho 4 kết quả match nhất